# Test the performance of the pre-trained GNN model for the problem of *Graph Coloring*

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
__counter__ = random.randint(0,2e9)

from IPython.display import HTML, display

In [3]:
import sys, os, shutil, json

gnn_path = "../classification/GNN/"
# adding GNN folder to the system path
sys.path.insert(0, gnn_path)

from train import testing
from data_loader_c import dataset_processing
from transfer_model import re_training, tune_parameters

Device is: cuda:0
Device is: cuda:0


A helper function to ensure that the dataset is made from scratch when NN is tested. 

In [4]:
def delete_folder_contents(folders):
    for folder in folders:
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

### Test set from a graph coloring dataset

Previously, we have trained a Graph Transformer model that solves the SAT-3 problem with all metrics > 90%. Now, we will use the pre-trained Graph Transformer model in order to solve the satisfiability of the graph-coloring problem as a classification tast. <br>

All NP-complete problems can be easily transformed to one another using algorithms of polynomial-time complexity. Let us give an example of how a 3-color graph-coloring problem can be transformed into a satisfiability one. <br>

Suppose we have the following graph that we want to check if it is 3-colorable i.e. every vertex will have a different color from its neighbors and the total number of colors that can be used is (less than) 3.

<img src="./plots/graph.jpg" height=100 width=200>

In order to transform this problem to a satisfiability one we follow the next steps by order:
<ol>
    <li>We assign <b>3 logic variables for every vertex</b> that represent whether this vertex is colored using the corresponding color. For example, for vertex $x_1$ we have: $x_{11}$ that is true if the vertex $x_1$ is colored using the first color and false otherwise, $x_{12}$ that is true if the vertex $x_1$ is colored using the second color and false otherwise and $x_{13}$ that is true if the vertex $x_1$ is colored using the third color and false otherwise. </li><br>
     <li> Given these new variables, we have to write the logical expressions that will be used to evaluate the colorability of the graph. More specifically,</li>
    <ul> 
        <li>A vertex should be colored with <b>at least one color</b>. For example, for vertex $x_1$ this statement translated to the following expression, $$x_{11} \vee x_{12} \vee x_{13}$$ </li> <br>
        <li>A vertex <b>cannot be colored with all 3 colors at the same time</b>. For example, for vertex $x_1$ this statement translated to the following expressions, $$\neg ( x_{11} \wedge x_{12}) \wedge \neg ( x_{11} \wedge x_{13}) \wedge \neg ( x_{12} \wedge x_{13})$$ that is equal to: (using De'Morgan law) $$ ( \neg x_{11} \vee \neg x_{12}) \wedge ( \neg x_{11} \vee \neg x_{13}) \wedge ( \neg x_{12} \vee \neg x_{13})$$</li> <br>
         <li>A vertex <b>cannot be colored with the same color as its neighbor</b>. For example, for vertex $x_1$ this statement translated to the following expressions, $$\neg ( x_{11} \wedge x_{21}) \wedge \neg ( x_{12} \wedge x_{22}) \wedge \neg ( x_{13} \wedge x_{23})$$ that is equal to: (using De'Morgan law) $$ ( \neg x_{11} \vee \neg x_{21}) \wedge ( \neg x_{12} \vee \neg x_{22}) \wedge ( \neg x_{13} \vee \neg x_{23})$$</li> <br>
    </ul> 
    <li>Since all these expressions must hold true for all variables, combine them using the logical-and in order to create a <b>CNF clause</b>. This final CNF is now the represenation of the graph coloring problem as a satisfiablity one. It is not a 3-SAT one by nature, but we can make it one. For more details please refer to the report.
    </li><br>
    
</ol>

Erase the previous dataset created by PyTorch

In [5]:
delete_folder_contents(["./raw", "./processed"])

**Create the dataset**: The dataset is created from the raw data in such a format that it can be loader by the *DataLoader* module of torch.geometric. The dataset is created in a same manner as the dataset in the SAT-3 problem. <br>

It should be noted that the original dataset only consists of satisfiable instances. ***In order to train the algorithm using also negative examples a simple idea was applied: In every positive instance we randomly select 4 vertices and create a clique of size 4. That instantly makes the graph not 3-colorable.***

In [6]:
pos_weight = dataset_processing()

Start the data processing...

Satisfiable CNFs   : 1699
Unsatisfiable CNFs : 1699

Ratio of SAT   : 0.5000
Ratio of UNSAT : 0.5000

Training set size: 2718
Test set size: 680
Dataset size: 3398

Processing completed.


<b>Get the tuned parameters of the pre-trained model.</b> 

In [7]:
# Load the parameters of the pre-trained GNN model
with open(gnn_path+'best_parameters_same_sets.txt') as f:
    data = f.read()

best_pre_trained_parameters = json.loads(data)

**Test the performance of the model *without* transfer learning**.

In [8]:
print("Test before retraining...\n")
testing(params=best_pre_trained_parameters, model_name=gnn_path+'final_model_same_sets.pth')

Test before retraining...

Dataset loading...


Processing...
100%|███████████████████████████████████████████████████████████████████████████████| 680/680 [00:01<00:00, 563.23it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed


Test set metrics:

 Confusion matrix: 
 [[241 205]
 [124 110]]
F1 Score  : 0.4007
Accuracy  : 0.5162
Precision : 0.4701
Recall    : 0.3492
ROC AUC   : 0.5047
Test Loss : 0.6083021055568348


In theory, these two problems are very similar, however in terms of modelling for a Graph Transformer they are quite different, as noted from the above results. Thus, **using the pre-trained Graph Transformer model, we will apply transfer learning**. For more information about the process, please refer to the report.

## Training a Graph Transformer classifier for the Graph Coloring problem using Transfer Learning

**Tune parameters**: Tune some parameters regarding the *non-frozen layers of the model* as well as parameters such as *learning rate etc*.

In [9]:
# Tune the algo after applying transfer learning
best_parameters = tune_parameters(pos_weight=pos_weight,  model_name=gnn_path+'final_model_same_sets.pth',
                                  best_pre_trained=best_pre_trained_parameters)


Test number 0 | Start testing new parameter-combination...

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 2718/2718 [00:05<00:00, 534.39it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.7227
Validation Loss : 0.6935

EPOCH | 1
Training Loss   : 0.6959
Validation Loss : 0.6937

EPOCH | 2
Training Loss   : 0.6931
Validation Loss : 0.6946

EPOCH | 3
Training Loss   : 0.6920
Validation Loss : 0.6953

EPOCH | 4
Training Loss   : 0.6914
Validation Loss : 0.6960

EPOCH | 5
Training Loss   : 0.6910
Validation Loss : 0.6966

EPOCH | 6
Training Loss   : 0.6908
Validation Loss : 0.6972

EPOCH | 7
Training Loss   : 0.6905
Validation Loss : 0.6978

EPOCH | 8
Training Loss   : 0.6904
Validation Loss : 0.6983

EPOCH | 9
Training Loss   : 0.6902
Validation Loss : 0.6989

EPOCH | 10
Training Loss   : 0.6901
Validation Loss : 0.6994

EPOCH | 11
Training Loss   : 0.6900
Validation Loss : 0.6999

EPOCH | 12
Training Loss   : 0.6899
Validation Loss : 0.7003

EPOCH | 13
Training Loss   : 0.6898
Validation Loss : 0.7008

EPOCH | 14
Training Loss   : 0.6898
Validation Loss : 0.7012

EPOCH | 15


Training Loss   : 0.6931
Validation Loss : 0.8050

EPOCH | 7
Training Loss   : 0.6926
Validation Loss : 0.8131

EPOCH | 8
Training Loss   : 0.6921
Validation Loss : 0.8196

EPOCH | 9
Training Loss   : 0.6916
Validation Loss : 0.8251

EPOCH | 10
Training Loss   : 0.6912
Validation Loss : 0.8299

EPOCH | 11
Training Loss   : 0.6908
Validation Loss : 0.8341

EPOCH | 12
Training Loss   : 0.6904
Validation Loss : 0.8380

EPOCH | 13
Training Loss   : 0.6900
Validation Loss : 0.8414

EPOCH | 14
Training Loss   : 0.6897
Validation Loss : 0.8445

EPOCH | 15
Training Loss   : 0.6895
Validation Loss : 0.8472

EPOCH | 16
Early stopping activated, with training and validation loss difference: 0.0081

Test number 7 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.7032
Validation Loss : 0.7108

EPOCH | 1
Training Loss   : 0.6964
Validation Loss : 0.7279

EPOCH | 2
Training Loss   : 0.695

Training Loss   : 0.6887
Validation Loss : 0.7151

EPOCH | 12
Training Loss   : 0.6886
Validation Loss : 0.7158

EPOCH | 13
Training Loss   : 0.6885
Validation Loss : 0.7166

EPOCH | 14
Training Loss   : 0.6884
Validation Loss : 0.7173

EPOCH | 15
Training Loss   : 0.6883
Validation Loss : 0.7180

EPOCH | 16
Early stopping activated, with training and validation loss difference: 0.0099

Test number 13 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.7094
Validation Loss : 0.6995

EPOCH | 1
Training Loss   : 0.6914
Validation Loss : 0.7022

EPOCH | 2
Training Loss   : 0.6905
Validation Loss : 0.7037

EPOCH | 3
Training Loss   : 0.6900
Validation Loss : 0.7054

EPOCH | 4
Training Loss   : 0.6897
Validation Loss : 0.7070

EPOCH | 5
Training Loss   : 0.6894
Validation Loss : 0.7084

EPOCH | 6
Training Loss   : 0.6892
Validation Loss : 0.7095

EPOCH | 7
Training Loss   : 0.6891

In [10]:
# Show best parameters
print(f'Best hyperparameters were: {best_parameters}')
# Store best parameters
with open('./best_parameters_same_sets.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()

Best hyperparameters were: {'batch_size': 16, 'learning_rate': 0.001, 'weight_decay': 0.001, 'pos_weight': 1.0, 'model_embedding_size': 64, 'model_attention_heads': 1, 'model_layers': 1, 'model_dropout_rate': 0.1, 'model_dense_neurons': 128}


**Re-train using the optimal parameters.**

In [11]:
# Access the best parameters in order to train final model
with open('best_parameters_same_sets.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

# Do the transfer learning with the optimal parameters
re_training(params=best_parameters_loaded, best_pre_trained_params=best_pre_trained_parameters,
            model_name=gnn_path+'final_model_same_sets.pth')

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.7227
Validation Loss : 0.6934

EPOCH | 1
Training Loss   : 0.6959
Validation Loss : 0.6934

EPOCH | 2
Training Loss   : 0.6931
Validation Loss : 0.6940

EPOCH | 3
Training Loss   : 0.6920
Validation Loss : 0.6944

EPOCH | 4
Training Loss   : 0.6914
Validation Loss : 0.6948

EPOCH | 5
Training Loss   : 0.6911
Validation Loss : 0.6951

EPOCH | 6
Training Loss   : 0.6908
Validation Loss : 0.6955

EPOCH | 7
Training Loss   : 0.6906
Validation Loss : 0.6958

EPOCH | 8
Training Loss   : 0.6904
Validation Loss : 0.6961

EPOCH | 9
Training Loss   : 0.6902
Validation Loss : 0.6964

EPOCH | 10
Training Loss   : 0.6901
Validation Loss : 0.6967

EPOCH | 11
Training Loss   : 0.6900
Validation Loss : 0.6970

EPOCH | 12
Training Loss   : 0.6899
Validation Loss : 0.6973

EPOCH | 13
Training Loss   : 0.6899
Validation Loss : 0.6975

EPOCH | 14
Training Loss   : 0.6898
Validation Loss : 

0.693978653234594

**Test the performance of the model *after* transfer learning.**

In [12]:
print("Test after retraining...\n")
testing(params=best_parameters_loaded, model_name=gnn_path+'final_model_same_sets_c.pth')

Test after retraining...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed


Test set metrics:

 Confusion matrix: 
 [[194 148]
 [171 167]]
F1 Score  : 0.5115
Accuracy  : 0.5309
Precision : 0.4941
Recall    : 0.5302
ROC AUC   : 0.5308
Test Loss : 0.6956874159879463


The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [14]:
print("\n1.")
display(HTML('<img src="plots/cm_same.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc_same.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr_same.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.


Unfortunatelly, the amount of data that we have does not make it possible to re-train more layers of the GTN and achieve better results.